---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

if False:
    def blight_model():
        train = loadcsv(train.csv)
    # Your code here
    
        return # Your answer here

In [54]:
if False:
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    train.head()

In [56]:
if False:
    test.head()

In [5]:
train_new = train.dropna(subset = ['compliance'])

In [6]:
train_new.shape

(159880, 34)

In [7]:
address = pd.read_csv('addresses.csv')

In [8]:
latlons = pd.read_csv('latlons.csv')

In [9]:
address.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [9]:
Complete_address= address.set_index('address').join(latlons.set_index('address'),how = 'left')

In [10]:
train_data= train_new.set_index('ticket_id').join(Complete_address.set_index('ticket_id'))

In [11]:
train_data.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,42.326937,-83.135118
22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,CA,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.145257,-83.208233
18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,MI,...,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.433466,-83.023493
18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,MI,...,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.388641,-83.037858


In [12]:
test_data = test.set_index('ticket_id').join(Complete_address.set_index('ticket_id'))

In [13]:
train_data.shape

(159880, 35)

In [14]:
test_data.shape

(61001, 28)

In [15]:
list_to_remove_train = ['balance_due','collection_status','compliance_detail','payment_amount','payment_date','payment_status']

In [16]:
train_data.drop(list_to_remove_train,axis = 1, inplace = True)

In [17]:
train_y = train_data['compliance']

In [18]:

train_data.drop(['compliance'],axis = 1, inplace = True)

In [19]:
list_to_remove_all = ['agency_name','state','disposition','violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
train_data.drop(list_to_remove_all,axis = 1, inplace = True)

In [20]:
train_data.lat.fillna(method='pad', inplace=True)
train_data.lon.fillna(method='pad', inplace=True)
test_data.lat.fillna(method='pad', inplace=True)
test_data.lon.fillna(method='pad', inplace=True)


In [21]:
train_X = train_data

In [22]:
train_X.head()

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,lat,lon
ticket_id,,,,,,,,,
22056,250.0,20.0,10.0,25.0,0.0,0.0,305.0,42.390729,-83.124268
27586,750.0,20.0,10.0,75.0,0.0,0.0,855.0,42.326937,-83.135118
22046,250.0,20.0,10.0,25.0,0.0,0.0,305.0,42.145257,-83.208233
18738,750.0,20.0,10.0,75.0,0.0,0.0,855.0,42.433466,-83.023493
18735,100.0,20.0,10.0,10.0,0.0,0.0,140.0,42.388641,-83.037858


In [23]:
from sklearn.naive_bayes import GaussianNB
from adspy_shared_utilities import plot_class_regions_for_classifier

In [24]:
nbclf = GaussianNB().fit(train_X, train_y)

In [25]:
test_data.drop(list_to_remove_all,axis = 1, inplace = True)

In [26]:
test_y = nbclf.predict_proba(test_data)

In [33]:
test_y[:,0].min()

0.0

In [43]:
test_data['compliance'] = test_y[:,1]
test_data.set_index('ticket_id', inplace=True)


KeyError: 'ticket_id'

In [38]:
#test_data.drop(['y'], axis = 1, inplace =True)

ValueError: labels ['y'] not contained in axis

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot

In [35]:
clf = RandomForestClassifier().fit(train_X, train_y)


In [40]:
test_y_rdfr = clf.predict_proba(test_data.dropna())

In [41]:
test_y_rdfr

array([[ 0.9 ,  0.1 ],
       [ 1.  ,  0.  ],
       [ 0.95,  0.05],
       ..., 
       [ 1.  ,  0.  ],
       [ 1.  ,  0.  ],
       [ 1.  ,  0.  ]])

In [52]:
def blight_model():
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    Complete_address= address.set_index('address').join(latlons.set_index('address'),how = 'left')
    train_data= train_new.set_index('ticket_id').join(Complete_address.set_index('ticket_id'))
    test_data = test.set_index('ticket_id').join(Complete_address.set_index('ticket_id'))
    list_to_remove_train = ['balance_due','collection_status','compliance_detail','payment_amount','payment_date','payment_status']
    train_data.drop(list_to_remove_train,axis = 1, inplace = True)
    train_y = train_data['compliance']
    train_data.drop(['compliance'],axis = 1, inplace = True)
    list_to_remove_all = ['agency_name','state','disposition','violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
    train_data.drop(list_to_remove_all,axis = 1, inplace = True)
    train_data.lat.fillna(method='pad', inplace=True)
    train_data.lon.fillna(method='pad', inplace=True)
    test_data.lat.fillna(method='pad', inplace=True)
    test_data.lon.fillna(method='pad', inplace=True)
    train_X = train_data
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    train_X_s = scaler.fit_transform (train_X)
    test_data.drop(list_to_remove_all,axis = 1, inplace = True)

    test_X_s =scaler.transform(test_data)
    clf = MLPClassifier(hidden_layer_sizes = [100,10], alpha = 5, random_state = 0, solver = 'lbfgs', verbose = 0).fit(train_X_s, train_y)
    test_y = clf.predict_proba(test_X_s)
    test_data['compliance'] = test_y[:,1]
    # Your code here

    
    return test_data['compliance']

In [53]:
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.052242
285362    0.027421
285361    0.061006
285338    0.052123
285346    0.062632
285345    0.052138
285347    0.068690
285342    0.310419
285530    0.026914
284989    0.038414
285344    0.068746
285343    0.026959
285340    0.026969
285341    0.068667
285349    0.062614
285348    0.052126
284991    0.038413
285532    0.038188
285406    0.038671
285001    0.038219
285006    0.026946
285405    0.027443
285337    0.038899
285496    0.068417
285497    0.051775
285378    0.027415
285589    0.038966
285585    0.050616
285501    0.062368
285581    0.027427
            ...   
376367    0.032448
376366    0.046667
376362    0.046729
376363    0.051332
376365    0.032448
376364    0.046667
376228    0.047673
376265    0.046264
376286    0.359369
376320    0.047373
376314    0.047978
376327    0.344734
376385    0.328825
376435    0.691981
376370    0.343167
376434    0.068499
376459    0.063001
376478    0.001353
376473    0.047684
376484    0.044030
376482    0.038538
37

In [49]:
def blight_model1():
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    Complete_address= address.set_index('address').join(latlons.set_index('address'),how = 'left')
    train_data= train_new.set_index('ticket_id').join(Complete_address.set_index('ticket_id'))
    test_data = test.set_index('ticket_id').join(Complete_address.set_index('ticket_id'))
    list_to_remove_train = ['balance_due','collection_status','compliance_detail','payment_amount','payment_date','payment_status']
    train_data.drop(list_to_remove_train,axis = 1, inplace = True)
    train_y = train_data['compliance']
    train_data.drop(['compliance'],axis = 1, inplace = True)
    list_to_remove_all = ['agency_name','state','disposition','violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
    train_data.drop(list_to_remove_all,axis = 1, inplace = True)
    train_data.lat.fillna(method='pad', inplace=True)
    train_data.lon.fillna(method='pad', inplace=True)
    test_data.lat.fillna(method='pad', inplace=True)
    test_data.lon.fillna(method='pad', inplace=True)
    train_X = train_data
    from sklearn.naive_bayes import GaussianNB
    from adspy_shared_utilities import plot_class_regions_for_classifier
    nbclf = GaussianNB().fit(train_X, train_y)
    test_data.drop(list_to_remove_all,axis = 1, inplace = True)
    test_y = nbclf.predict_proba(test_data)
    test_data['compliance'] = test_y[:,1]
    # Your code here

    
    return test_data['compliance']